In [ ]:



import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from glob import glob

import librosa
import librosa.display
import IPython.display as ipd

from itertools import cycle

sns.set_theme(style="white", palette=None)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])


In [10]:
audio_files = glob('my_song.wav')
print(f"Found {audio_files} audio files.")

Found ['my_song.wav'] audio files.


In [13]:
ipd.Audio(audio_files[0])
